In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install html2text
!pip install datefinder
!pip install -U dask[complete]

In [ ]:
import dask.dataframe as dd
import dask
import pandas as pd
from dask.distributed import Client
from news_parser import filter_body, time, body_formatter

In [ ]:
# client = Client(memory_limit='25GB', processes=False,
#                 n_workers=2, threads_per_worker=1)
# client

In [ ]:
dask.config.set(scheduler="threads")

In [ ]:
input_dir = "data/raw_bzg/"
output_dir = 'data/unraw1_bzg/'

### First, strip documents of html, otherwise html_body cells are too large to handle for pyarrow.

In [ ]:
# for year in range(2019, 2020):
#     print(year)
#     df = pd.read_parquet(f"{input_dir}story_df_raw_{year}.parquet")
#     df = dd.from_pandas(df, npartitions=12)
#     df["html_body"] = df["html_body"].apply(body_formatter, meta=pd.Series(dtype="str"))
#     df = df.rename(columns={"html_body":"body"})
#     name_function = lambda x: f"data-{year}-{x}.parquet"
#     df.to_parquet(output_dir, name_function=name_function)

### Repartition



In [54]:
input_dir = 'data/unraw1_bzg/'
output_dir = 'data/unraw2_bzg/'

In [61]:
ddf = dd.read_parquet(input_dir+"*.parquet")

In [65]:
ddf2 = ddf.repartition(npartitions=50)

In [66]:
name_function = lambda x: f"data-{x}.parquet"
ddf2.to_parquet(output_dir, name_function=name_function)

In [76]:
list_of_authors = ddf.author.unique()

,time,stocks,author,title,channels,body
34016421,2023-08-23 21:51:18-04:00,KEX,Globe Newswire,Boat Christening Ceremony to Celebrate Nation'...,"[""Press Releases""]","HOUSTON, Aug. 23, 2023 (GLOBE NEWSWIRE) --\n\n..."
34016619,2023-08-23 22:43:00-04:00,CD,PRNewswire,Chindata Group Unveils Pioneering Full-Stack S...,"[""Press Releases""]","BEIJING, Aug. 23, 2023 /PRNewswire/ -- Chindat..."
34016667,2023-08-23 23:00:00-04:00,OIG,PRNewswire,Orbital Infrastructure Group Inc. Intends to E...,"[""Commodities"", ""Press Releases""]",_Receives Commitment from Existing Secured Len...
34016713,2023-08-23 23:11:00-04:00,IBRX,PRNewswire,"IBRX FINAL DEADLINE TUESDAY: ROSEN, TRUSTED IN...","[""Press Releases""]","NEW YORK, Aug. 23, 2023 /PRNewswire/ --\n\n**W..."
34017843,2023-08-23 23:06:54-04:00,TSX:OTEX,WebWire,OpenText Buys KineMatik,"[""News"", ""Press Releases""]","OpenText™ï¿½(NASDAQ:ï¿½OTEX), (TSX:OTEX) annou..."
